<a href="https://colab.research.google.com/github/Sejong-Kaggle-Challengers-2nd/Cho-D-YoungRae/blob/main/6/2%EB%93%B1%EC%BD%94%EB%93%9C%EA%B3%B5%EB%B6%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [제 2회 컴퓨터 비전 학습 경진대회](https://dacon.io/competitions/official/235697/overview/description/)
> [2등 코드](https://dacon.io/competitions/official/235697/codeshare/2452?page=1&dtype=recent)

## Study
1. EfficientNet
2. swish activation
3. SAM Optimizer
4. pytorch pretrained model 의 normalize에 대한 필요
5. CosineAnnealingWarmRestart (LRScheduler)

In [1]:
!unzip /content/drive/MyDrive/kaggle-study/6/data.zip

Archive:  /content/drive/MyDrive/kaggle-study/6/data.zip
  inflating: dirty_mnist_2nd.zip     
  inflating: dirty_mnist_2nd_answer.csv  
  inflating: mnist_data.zip          
  inflating: sample_submission.csv   
  inflating: test_dirty_mnist_2nd.zip  


In [2]:
from google.colab import output
# 현재 디렉터리에 dirty_mnist라는 폴더 생성
!mkdir "./dirty_mnist"
#dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
!unzip "/content/dirty_mnist_2nd.zip" -d "./dirty_mnist_2nd/"
# 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist"
#test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "/content/test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist_2nd/"
# 출력 결과 지우기
output.clear()


전처리 : 
Rotation augmentation만 적용 
Flip을 적용할 시 알파벳 생김새가 겹칠 수 있다는 생각을 했습니다. (u <->n 등)

학습: 
EfficientNet 계열의 backbone을 사용하여 학습했습니다.
Ensemble을 할 모델들에게 최대한의 많은 다양성을 주기 위해 여러 세팅을 번갈아가며 돌렸습니다.

Backbone: B3 ~ B7
Learning rate scheduler: None or CosineAnnealingWithWarmRestarts 
Optimizer: AdamW or SAM
MixUp: None or Used
Initial Lr : 1e-3, 1e-4...

Early stopping을 사용하였습니다.
Loss/accuracy중 하나만 향상되더라도 early stopping의 count를 리셋하였습니다.

EfficientNet 의 어떤 모델을 사용하느냐에 따라 GPU메모리에 다 들어가지 않는 경우도 많아서 batch size를 조정하면서 진행했습니다.
Single-GPU, TITAN XP를 사용하였습니다.

단순하게 train set과 valid set을 고정해두고 진행했습니다.

후처리:
Ensemble을 통하여 결과를 낸 것 외에 별다른 후처리는 없었습니다.

최종 사용한 모델들의 backbone network를 제외하고는 정확한 트레이닝 scheme은 기억나지 않지만, 
pretrained weight들이 있어 재현은 가능합니다.

코드는 우주대마왕님께서 공유해주신 baseline code에서 크게 벗어나지 않습니다. https://dacon.io/competitions/official/235697/codeshare/2354?dtype=recent 반복되는 설명은 생략하겠습니다.

In [3]:
!pip install torchinfo
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=7262a395f07179a3a71dd69c81c21862a82838f5caf3c976b109b108034ac568
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch


In [4]:
import os
import random
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import albumentations as A
# from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts # 이건줄 알았는데 아님

from torchinfo import summary

from torchvision import transforms
from torch.autograd import Variable

# from model import * # 아래 설명 확인
from efficientnet_pytorch import EfficientNet

import math
from torch.optim.lr_scheduler import _LRScheduler

여기서 model이란 하단과 같은 efficientnet backbone을 의미합니다. efficientnetb3부터 b7까지 사용하였습니다. 모두 비슷한 구조를 띄고 있기에 예시로 두개만 첨부하였습니다.
> `model`: 대회 출전자가 직접 정의한 모델이 들어있는 파일인듯

[EfficientNet 설명](https://hoya012.github.io/blog/EfficientNet-review/)
> width, depth, resolution 을 적절하게 scaling해줘서 가벼운 모델임에도 높은 성능을 낼 수 있게 하였다.

[swish 및 몇가지 activation](https://eehoeskrap.tistory.com/440)

In [5]:
class MnistModel_efficientb3(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.backbone = EfficientNet.from_pretrained('efficientnet-b3')

        self.backbone._fc = nn.Linear(1536, 512)
        self.dropout = nn.Dropout(p=0.2)
        self.activation = self.backbone._swish  # efficientnet activation
        self.classifier = nn.Linear(512, 26)

    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        x = self.activation(x)
        x = self.classifier(x)

        return x

class MnistModel_efficientb4(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.backbone = EfficientNet.from_pretrained('efficientnet-b4')

        self.backbone._fc = nn.Linear(1792, 512)
        self.dropout = nn.Dropout(p=0.2)
        self.activation = self.backbone._swish
        self.classifier = nn.Linear(512, 26)

    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        x = self.activation(x)
        x = self.classifier(x)

        return x

dacon 코드공유에 나와있지 않은 나머지 b5~b7

In [6]:
class MnistModel_efficientb5(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.backbone = EfficientNet.from_pretrained('efficientnet-b5')

        self.backbone._fc = nn.Linear(2048, 512)
        self.dropout = nn.Dropout(p=0.2)
        self.activation = self.backbone._swish
        self.classifier = nn.Linear(512, 26)

    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        x = self.activation(x)
        x = self.classifier(x)

        return x


class MnistModel_efficientb6(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.backbone = EfficientNet.from_pretrained('efficientnet-b6')

        self.backbone._fc = nn.Linear(2304, 512)
        self.dropout = nn.Dropout(p=0.2)
        self.activation = self.backbone._swish
        self.classifier = nn.Linear(512, 26)

    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        x = self.activation(x)
        x = self.classifier(x)

        return x


class MnistModel_efficientb7(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.backbone = EfficientNet.from_pretrained('efficientnet-b7')

        self.backbone._fc = nn.Linear(2560, 512)
        self.dropout = nn.Dropout(p=0.2)
        self.activation = self.backbone._swish
        self.classifier = nn.Linear(512, 26)

    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        x = self.activation(x)
        x = self.classifier(x)

        return x

DACON.JIN 님께서 소개해주신 SAM Optimizer를 사용하였습니다. https://dacon.io/competitions/official/235697/codeshare/2370?page=2&dtype=recent 그 외에는 기본적인 AdamW를 사용하였습니다.

In [7]:
class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                torch.stack([
                  p.grad.norm(p=2).to(shared_device)
                    for group in self.param_groups for p in group["params"]
                      if p.grad is not None
                    ]),
                    p=2
               )
        return norm


시드를 고정하고, 데이터셋을 정의하고, loader를 불러오는 과정까지 제가 참고한 baseline code에서 크게 벗어나지 않습니다. 다만, 알파벳을 분류하는 현 문제의 경우 flip augmentation은 적절하지 않다고 느꼈습니다. (u <-> n, Z <-> S 로 헷갈릴 수 있을 것 같다고 판단). 이에 따라 Flip을 제외하고 rotation augmentation만 사용하였습니다. 정확히 기억은 나지 않지만 유의미한 성능 향상이 있던 것으로 기억합니다.

[normalize 사용이유](https://pytorch.org/vision/stable/models.html)
> All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 224. The images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]. You can use the following transform to normalize:
```python
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
```

np.random.shuffle 로 index 섞어 줬는데 또 SubsetRandomSampler 사용하는 이유가 뭘까 -> 사용하지 않아도 되지 않을까...

In [8]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(42)

class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:])) # 0: index

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

# normalize
transforms_train = transforms.Compose([
    #transforms.RandomHorizontalFlip(p=0.5),
    #transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(180, expand=False),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

trainset = MnistDataset('./dirty_mnist_2nd', './dirty_mnist_2nd_answer.csv', transforms_train)

dataset_size = len(trainset)
validation_split = 0.1
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, sampler=train_sampler, num_workers=8)
validation_loader = torch.utils.data.DataLoader(trainset, batch_size=16, sampler=valid_sampler, num_workers=4)
# test_loader = DataLoader(testset, batch_size=16, num_workers=4)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel_efficientb4().to(device)
print(summary(model, input_size=(1, 3, 256, 256), verbose=0))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/ef


Loaded pretrained weights for efficientnet-b4
Layer (type:depth-idx)                                  Output Shape              Param #
├─EfficientNet: 1-1                                     [1, 512]                  --
|    └─Conv2dStaticSamePadding: 2-1                     [1, 48, 128, 128]         --
|    |    └─ZeroPad2d: 3-1                              [1, 3, 257, 257]          --
|    └─BatchNorm2d: 2-2                                 [1, 48, 128, 128]         96
|    └─MemoryEfficientSwish: 2-3                        [1, 48, 128, 128]         --
├─MemoryEfficientSwish: 1-2                             [1, 48, 128, 128]         --
├─EfficientNet: 1                                       []                        --
|    └─ModuleList: 2                                    []                        --
|    |    └─MBConvBlock: 3-2                            [1, 24, 128, 128]         2,940
|    |    └─MBConvBlock: 3-3                            [1, 24, 128, 128]         1,206
|    | 

하단에 AdamW와 SAM을 번갈아가며 사용한 흔적이 남아있습니다. 또한 scheduler를 사용하지 않다가 (1e-3 lr 에 고정), 후반부에 CosineAnnealingWithWarmRestarts를 적용하였습니다. 이 또한 유의미한 성능 향상이 있었습니다. 보편적인 cosine annealing과 다르게, 매 iteration이 아닌 매 epoch마다 step을 적용하였습니다.



사용된 CosineAnnealingWarmRestarts 는 다음과 같습니다. https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup/blob/master/cosine_annearing_with_warmup.py
> [설명해놓은 블로그 글](https://gaussian37.github.io/dl-pytorch-lr_scheduler/#cosineannealingwarmrestarts-1)



In [9]:
class CosineAnnealingWarmupRestarts(_LRScheduler):
    """
        optimizer (Optimizer): Wrapped optimizer.
        first_cycle_steps (int): First cycle step size.
        cycle_mult(float): Cycle steps magnification. Default: -1.
        max_lr(float): First cycle's max learning rate. Default: 0.1.
        min_lr(float): Min learning rate. Default: 0.001.
        warmup_steps(int): Linear warmup step size. Default: 0.
        gamma(float): Decrease rate of max learning rate by cycle. Default: 1.
        last_epoch (int): The index of last epoch. Default: -1.
    """
    
    def __init__(self,
                 optimizer : torch.optim.Optimizer,
                 first_cycle_steps : int,
                 cycle_mult : float = 1.,
                 max_lr : float = 0.1,
                 min_lr : float = 0.001,
                 warmup_steps : int = 0,
                 gamma : float = 1.,
                 last_epoch : int = -1
        ):
        assert warmup_steps < first_cycle_steps
        
        self.first_cycle_steps = first_cycle_steps # first cycle step size
        self.cycle_mult = cycle_mult # cycle steps magnification
        self.base_max_lr = max_lr # first max learning rate
        self.max_lr = max_lr # max learning rate in the current cycle
        self.min_lr = min_lr # min learning rate
        self.warmup_steps = warmup_steps # warmup step size
        self.gamma = gamma # decrease rate of max learning rate by cycle
        
        self.cur_cycle_steps = first_cycle_steps # first cycle step size
        self.cycle = 0 # cycle count
        self.step_in_cycle = last_epoch # step size of the current cycle
        
        super(CosineAnnealingWarmupRestarts, self).__init__(optimizer, last_epoch)
        
        # set learning rate min_lr
        self.init_lr()
    
    def init_lr(self):
        self.base_lrs = []
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.min_lr
            self.base_lrs.append(self.min_lr)
    
    def get_lr(self):
        if self.step_in_cycle == -1:
            return self.base_lrs
        elif self.step_in_cycle < self.warmup_steps:
            return [(self.max_lr - base_lr)*self.step_in_cycle / self.warmup_steps + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.max_lr - base_lr) \
                    * (1 + math.cos(math.pi * (self.step_in_cycle-self.warmup_steps) \
                                    / (self.cur_cycle_steps - self.warmup_steps))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.step_in_cycle = self.step_in_cycle + 1
            if self.step_in_cycle >= self.cur_cycle_steps:
                self.cycle += 1
                self.step_in_cycle = self.step_in_cycle - self.cur_cycle_steps
                self.cur_cycle_steps = int((self.cur_cycle_steps - self.warmup_steps) * self.cycle_mult) + self.warmup_steps
        else:
            if epoch >= self.first_cycle_steps:
                if self.cycle_mult == 1.:
                    self.step_in_cycle = epoch % self.first_cycle_steps
                    self.cycle = epoch // self.first_cycle_steps
                else:
                    n = int(math.log((epoch / self.first_cycle_steps * (self.cycle_mult - 1) + 1), self.cycle_mult))
                    self.cycle = n
                    self.step_in_cycle = epoch - int(self.first_cycle_steps * (self.cycle_mult ** n - 1) / (self.cycle_mult - 1))
                    self.cur_cycle_steps = self.first_cycle_steps * self.cycle_mult ** (n)
            else:
                self.cur_cycle_steps = self.first_cycle_steps
                self.step_in_cycle = epoch
                
        self.max_lr = self.base_max_lr * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [10]:
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
#optimizer = SAM(model.parameters(), base_optimizer, lr=1e-3)
scheduler = CosineAnnealingWarmupRestarts(optimizer,
                                          first_cycle_steps=60,
                                          cycle_mult=1.0,
                                          max_lr=0.005,
                                          min_lr=0.0001,
                                          warmup_steps=12,
                                          gamma=1.0)
criterion = nn.MultiLabelSoftMarginLoss()

Mixup augmentation을 적용하기도 하였습니다. 다양한 모델을 모아 ensemble을 통해 결과를 내고자 하였기에, 다양한 세팅을 번갈아가며 사용했다는 점을 유의해주시면 되겠습니다.


In [ ]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

num_epochs = 60

best_loss = 1e10
best_acc = 0
no_improvement = 0
for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()

        images = images.to(device) 
        targets = targets.to(device)
        
        images, targets_a, targets_b, lam = mixup_data(images, targets)
        images, targets_a, targets_b = map(Variable, (images, targets_a, targets_b))

        outputs = model(images)
        loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)

        loss.backward()
        optimizer.step()

        #mixup_criterion(criterion, model(images), targets_a, targets_b, lam).backward()
        #optimizer.second_step(zero_grad=True)

        outputs = outputs > 0.5
        acc = (outputs == targets).float().mean()
        print(f'EPOCH: {epoch}/{num_epochs} | {i} / {len(train_loader)} | LOSS: {loss.item():.5f}, ACCURACY: {acc.item():.5f}')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
EPOCH: 8/60 | 1159 / 1407 | LOSS: 0.52393, ACCURACY: 0.76322
EPOCH: 8/60 | 1160 / 1407 | LOSS: 0.63169, ACCURACY: 0.67067
EPOCH: 8/60 | 1161 / 1407 | LOSS: 0.60855, ACCURACY: 0.54327
EPOCH: 8/60 | 1162 / 1407 | LOSS: 0.59099, ACCURACY: 0.53005
EPOCH: 8/60 | 1163 / 1407 | LOSS: 0.54081, ACCURACY: 0.77764
EPOCH: 8/60 | 1164 / 1407 | LOSS: 0.66936, ACCURACY: 0.60938
EPOCH: 8/60 | 1165 / 1407 | LOSS: 0.65728, ACCURACY: 0.61659
EPOCH: 8/60 | 1166 / 1407 | LOSS: 0.59657, ACCURACY: 0.52043
EPOCH: 8/60 | 1167 / 1407 | LOSS: 0.63970, ACCURACY: 0.66226
EPOCH: 8/60 | 1168 / 1407 | LOSS: 0.57525, ACCURACY: 0.53966
EPOCH: 8/60 | 1169 / 1407 | LOSS: 0.63812, ACCURACY: 0.56130
EPOCH: 8/60 | 1170 / 1407 | LOSS: 0.53093, ACCURACY: 0.51923
EPOCH: 8/60 | 1171 / 1407 | LOSS: 0.65953, ACCURACY: 0.61418
EPOCH: 8/60 | 1172 / 1407 | LOSS: 0.64449, ACCURACY: 0.56250
EPOCH: 8/60 | 1173 / 1407 | LOSS: 0.55677, ACCURACY: 0.53726
EPOCH: 8/60 | 1174 / 1407 | LOSS: 0.64633, ACCURA

코드의 나머지 부분인 validation입니다. 조잡하게 짜여진 early stopping 코드가 민망하네요. 전체 스크립트를 설명만 추가하여 첨부하였기에 그대로 돌리실 수 있을 것 같습니다.

In [ ]:
    losses = []
    accuracies = []
    for i, (images, targets) in enumerate(validation_loader):
        model.eval()
        with torch.no_grad():
            images = images.to(device)
            targets = targets.to(device)

            outputs = model(images)
            loss = criterion(outputs, targets)

            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
        print(f'EPOCH: {epoch}/{num_epochs} | {i} / {len(validation_loader)} | LOSS: {loss.item():.5f}, ACCURACY: {acc.item():.5f}')
        losses.append(loss.item())
        accuracies.append(acc.item())
    
    flag = 0
    if sum(losses) / len(losses) < best_loss:
        print("Best loss achieved at {}".format(sum(losses) / len(losses)))
        best_loss = sum(losses) / len(losses)
        torch.save(model.state_dict(), 'mixup_loss_model_effnetb4_nosam.pt')
        flag += 1
        no_improvement = 0

    if sum(accuracies) / len(accuracies) > best_acc:
        print("Best acc achieved at {}(Ignore outputs!)".format(sum(accuracies) / len(accuracies)))
        best_acc = sum(accuracies) / len(accuracies)
        torch.save(model.state_dict(), 'mixup_acc_model_effnetb4_nosam.pt')
        flag += 1
        no_improvement = 0
    
    if flag == 0:
        no_improvement += 1
    
    if no_improvement > 30:
        print("No improvement for 3 epochs, stopping")
        break

    scheduler.step()

print("Best acc achieved:", best_acc)
print("Best loss achieved:", best_loss)